In [1]:
# First, make sure you have the transformers library installed:
# pip install transformers accelerate torch # or tensorflow depending on your backend

from transformers import AutoModelForSequenceClassification, AutoTokenizer
 
PRETRAINED_MODEL_NAME = "gpt2"  
print(f"Loading tokenizer from: {PRETRAINED_MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
print("Tokenizer loaded successfully.")

# --- 3. Load the model ---
 
print(f"Loading model from: {PRETRAINED_MODEL_NAME}")
model = AutoModelForSequenceClassification.from_pretrained(PRETRAINED_MODEL_NAME)
print("Model loaded successfully.")
 
SAVE_PATH = "./gpt2-it-ticket-classifier" # Choose a meaningful local path

print(f"Saving model and tokenizer to: {SAVE_PATH}")
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
print("Model and tokenizer saved successfully!")

c:\Users\parth\OneDrive\Desktop\one\RouteIQ\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading tokenizer from: gpt2
Tokenizer loaded successfully.
Loading model from: gpt2


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully.
Saving model and tokenizer to: ./gpt2-it-ticket-classifier
Model and tokenizer saved successfully!


In [3]:
from huggingface_hub import HfApi
from dotenv import load_dotenv,find_dotenv
import os
load_dotenv()


True

In [4]:


api = HfApi(token=os.getenv("HF_TOKEN"))
api.upload_folder(
    folder_path="./gpt2-it-ticket-classifier",
    repo_id="parth1609/gpt2-it-ticket-classifier",
    repo_type="model",
)


model.safetensors: 100%|██████████| 498M/498M [05:18<00:00, 1.56MB/s]   


CommitInfo(commit_url='https://huggingface.co/parth1609/gpt2-it-ticket-classifier/commit/e37987b6e8b86e9f730bce9896594ffbb302cb43', commit_message='Upload folder using huggingface_hub', commit_description='', oid='e37987b6e8b86e9f730bce9896594ffbb302cb43', pr_url=None, repo_url=RepoUrl('https://huggingface.co/parth1609/gpt2-it-ticket-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='parth1609/gpt2-it-ticket-classifier'), pr_revision=None, pr_num=None)

In [18]:
import requests
import os

# --- Configuration ---
# Your model ID on Hugging Face Hub
MODEL_ID = "parth1609/gpt2-it-ticket-classifier"

# Your Hugging Face API token
# It's highly recommended to load this from an environment variable for security
# Make sure you've set HF_TOKEN="hf_YOUR_TOKEN_HERE" in your environment
# API_TOKEN = os.getenv("HF_TOKEN")
API_TOKEN = os.getenv("HF_TOKEN")

if not API_TOKEN:
    raise ValueError("Hugging Face API token not found. Please set the HF_TOKEN environment variable.")

# The Inference API URL for your model
API_URL = f"https://api-inference.huggingface.co/models/{MODEL_ID}"

# Headers for authentication
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/json" # Specify content type for JSON payload
}

# --- Function to send a query to the API ---
def query(payload):
    """
    Sends a query (input text) to the Hugging Face Inference API.

    Args:
        payload (dict): A dictionary containing the input data for the model.
                        For text models, it's typically {"inputs": "Your input text"}.

    Returns:
        dict: The JSON response from the API, containing the model's predictions.
    """
    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    return response.json()


In [9]:

sample = [
    "My laptop is not turning on after the latest update.",
    "I need help resetting my VPN password, it keeps failing.",
    "The software update caused my computer to crash.",
]


In [29]:
from transformers import pipeline
import torch
import json

# initialize pipeline with your model
pipe = pipeline(
    "text-generation",
    model=MODEL_ID,
    tokenizer=MODEL_ID,
    device=0 
)

# example ticket list
sample_tickets = [
    "My laptop is not turning on after the latest update.",
    "The VPN disconnects every time I upload files.",
    "Wi-Fi drops every morning at 9 AM in the office."
]

for ticket in sample_tickets:
    prompt = (
        """<|im_start|>system
    You are an IT support specialist. Given a ticket description, you will **only** output valid JSON **with no additional text**, following this exact schema:
    ```json
    {
    "Department": "IT",         // one of: IT, Customer Service, Sales, Billing
    "Priority": "High"          // one of: High, Medium, Low
}
        "<|im_end|>\n"
        "<|im_start|>user\n"
        "Classify this support ticket into Department and Priority:\n\n"
        f'""\n{ticket}\n""\n'
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
        """
        )

    # include generation settings
    # output = pipe(prompt, max_new_tokens=512, do_sample=False)
    # text = output[0]["generated_text"].strip()

    out = pipe(prompt, max_new_tokens=60)[0]["generated_text"]
    print(json.loads(out))  # or eval(out) if it’s valid Python dict
    # print(out)

    print("🔹 Ticket:")
    print(ticket)
    print("✅ Model Output:")
    print(text)
    print("-" * 60)


Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


JSONDecodeError: Expecting value: line 1 column 1 (char 0)